## Neural Networks with PSD features, theta and alpha bands only


In [1]:
#Import necessary libraries

import pandas as pd
import numpy as np
from scipy import signal
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle as pkl
import itertools 
import glob
from sklearn import svm 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, f1_score
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

%matplotlib inline 
# %matplotlib qt


In [2]:
# Import datasets
psd_df = pd.read_pickle("F:\EEG-data\pkl\\featureExtraction\PSD_only\PSD_df_12.pkl")
auc_df = pd.read_pickle("F:\EEG-data\pkl\\featureExtraction\AUC_only\AUC_df_12.pkl")
AP_df = pd.read_pickle("F:\EEG-data\pkl\\featureExtraction\AUC_PSD\AUC_PSD_df_12.pkl")

In [17]:
#Baseline model
def baseline_func(no_features):
    ANN_model = keras.Sequential([
    # keras.layers.Flatten(input_shape=(8,32)),
    keras.layers.Dense(no_features,input_dim=no_features,activation='relu'),
    keras.layers.Dense(no_features/2, activation='relu'),
    keras.layers.Dense(no_features/4, activation='relu'),
    keras.layers.Dense(3,activation='softmax')])

    ANN_model.compile(optimizer="adam",
                loss="categorical_crossentropy",
                metrics = ['accuracy'])
    return ANN_model
#Baseline model 2
def baseline_func2(no_features):
    ANN_model = keras.Sequential([
    # keras.layers.Flatten(input_shape=(8,32)),
    keras.layers.Dense(no_features,input_dim=no_features,activation='relu'),
    keras.layers.Dense(no_features/2, activation='relu'),
    keras.layers.Dense(3,activation='softmax')])

    ANN_model.compile(optimizer="adam",
                loss="categorical_crossentropy",
                metrics = ['accuracy'])
    return ANN_model

#Baseline model 3
def baseline_func3(no_features):
    ANN_model = keras.Sequential([
    keras.layers.Dense(no_features,input_dim=no_features,activation='relu'),
    keras.layers.Dense(no_features/2, activation='relu'),
    keras.layers.Dense(no_features/4, activation='relu'),
    keras.layers.Dense(no_features/8, activation='relu'),
    keras.layers.Dense(3,activation='softmax')])

    ANN_model.compile(optimizer="adam",
                loss="categorical_crossentropy",
                metrics = ['accuracy'])
    return ANN_model

#Baseline model 4
def baseline_func4(no_features):
    ANN_model = keras.Sequential([
    keras.layers.Dense(no_features,input_dim=no_features,activation='relu'),
    keras.layers.Dense(3,activation='softmax')])

    ANN_model.compile(optimizer="adam",
                loss="categorical_crossentropy",
                metrics = ['accuracy'])
    return ANN_model

#Cross Validation and Prediction
def apply_CV_predict(baseline_func,df,filename,savedir):
    #Split into training and test sets 
    class_names = ["C","H","W"]
    X = df.iloc[:,:-1].values.astype(np.float32)
    y = df['Music'].apply(class_names.index).values.astype(np.float32)
    sss = StratifiedShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for train_index, test_index in sss.split(X,y):
                X_train, X_test = X[train_index],X[test_index]
                y_train, y_test = y[train_index],y[test_index]
    no_features = X.shape[1]
    #Cross Validation 
    epochsList = list(range(1,20))
    mean_results = [] 
    kfold = StratifiedShuffleSplit(n_splits=10)
    for epoch in epochsList:
        estimator=KerasClassifier(build_fn=lambda:baseline_func(no_features),epochs=epoch,batch_size=5,verbose=0)
        result = np.mean(cross_val_score(estimator,X_train,y_train,cv=kfold,scoring="f1_macro"))
        mean_results.append(result)
    mean_results = np.array(mean_results)
    best_epoch = epochsList[np.argmax(mean_results)]
    #Test set evaluation
    final_estimator = KerasClassifier(build_fn=lambda:baseline_func(no_features),epochs=best_epoch,batch_size=5,verbose=0)
    final_estimator.fit(X_train,y_train)
    y_pred = final_estimator.predict(X_test)
    #F1 Score
    f1Score = f1_score(y_test,y_pred,average="macro")
    #Confusion matrix
    cm = confusion_matrix(y_test,y_pred,normalize='true')
    # fig,ax = plt.subplots()
    # ax.set_title(filename+'CM')
    # plot_confusion_matrix(final_estimator,X_test,y_test,labels=class_names,ax=ax,normalize='true')
    # plt.savefig(savedir+filename+'.png')
    # plt.close()
    return f1Score,cm

In [4]:
#Directories to save to
savedir = "F:\EEG-data\\music\ANN/"
PSD_filename = "PSD_"
AUC_filename = "AUC_"
AP_filename = "AP_"

PSD_f1Score,PSD_cm = apply_CV_predict(baseline_func,psd_df,PSD_filename,savedir)


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [5]:
AUC_f1Score,AUC_cm = apply_CV_predict(baseline_func,auc_df,AUC_filename,savedir)


In [6]:
AP_f1Score,AP_cm = apply_CV_predict(baseline_func,AP_df,AP_filename,savedir)

In [7]:
print("PSD F1 Score:",PSD_f1Score)
print("AUC F1 Score:",AUC_f1Score)
print("AP F1 Score:",AP_f1Score)

PSD F1 Score: 0.6643142683565907
AUC F1 Score: 0.6743949021854556
AP F1 Score: 0.6789624830244759


In [8]:
print(PSD_cm)
print(AUC_cm)
print(AP_cm)

[[0.39215686 0.47058824 0.1372549 ]
 [0.046875   0.8828125  0.0703125 ]
 [0.04040404 0.23232323 0.72727273]]
[[0.66666667 0.14705882 0.18627451]
 [0.2109375  0.6015625  0.1875    ]
 [0.14141414 0.08080808 0.77777778]]
[[0.52941176 0.34313725 0.12745098]
 [0.09375    0.8515625  0.0546875 ]
 [0.14141414 0.21212121 0.64646465]]


In [10]:
PSD_f1Score2,PSD_cm2 = apply_CV_predict(baseline_func2,psd_df,PSD_filename,savedir)
AUC_f1Score2,AUC_cm2 = apply_CV_predict(baseline_func2,auc_df,AUC_filename,savedir)
AP_f1Score2,AP_cm2 = apply_CV_predict(baseline_func2,AP_df,AP_filename,savedir)

In [11]:
print("PSD F1 Score:",PSD_f1Score2)
print("AUC F1 Score:",AUC_f1Score2)
print("AP F1 Score:",AP_f1Score2)

PSD F1 Score: 0.7306826986247931
AUC F1 Score: 0.6919137428305936
AP F1 Score: 0.7066052227342549


In [12]:
print(PSD_cm2)
print(AUC_cm2)
print(AP_cm2)

[[0.56862745 0.29411765 0.1372549 ]
 [0.0703125  0.84375    0.0859375 ]
 [0.09090909 0.13131313 0.77777778]]
[[0.6372549  0.21568627 0.14705882]
 [0.1953125  0.6953125  0.109375  ]
 [0.14141414 0.11111111 0.74747475]]
[[0.74509804 0.21568627 0.03921569]
 [0.140625   0.8203125  0.0390625 ]
 [0.21212121 0.24242424 0.54545455]]


In [14]:
PSD_f1Score3,PSD_cm3 = apply_CV_predict(baseline_func3,psd_df,PSD_filename,savedir)
AUC_f1Score3,AUC_cm3 = apply_CV_predict(baseline_func3,auc_df,AUC_filename,savedir)
AP_f1Score3,AP_cm3 = apply_CV_predict(baseline_func3,AP_df,AP_filename,savedir)

In [15]:
print("PSD F1 Score:",PSD_f1Score3)
print("AUC F1 Score:",AUC_f1Score3)
print("AP F1 Score:",AP_f1Score3)

PSD F1 Score: 0.6843504491935916
AUC F1 Score: 0.7010393334341062
AP F1 Score: 0.6875735109969682


In [16]:
print(PSD_cm3)
print(AUC_cm3)
print(AP_cm3)

[[0.49019608 0.37254902 0.1372549 ]
 [0.09375    0.8515625  0.0546875 ]
 [0.1010101  0.19191919 0.70707071]]
[[0.69607843 0.18627451 0.11764706]
 [0.1875     0.71875    0.09375   ]
 [0.16161616 0.15151515 0.68686869]]
[[0.67647059 0.16666667 0.15686275]
 [0.2421875  0.6484375  0.109375  ]
 [0.12121212 0.13131313 0.74747475]]


In [18]:
PSD_f1Score4,PSD_cm4 = apply_CV_predict(baseline_func4,psd_df,PSD_filename,savedir)
AUC_f1Score4,AUC_cm4 = apply_CV_predict(baseline_func4,auc_df,AUC_filename,savedir)
AP_f1Score4,AP_cm4 = apply_CV_predict(baseline_func4,AP_df,AP_filename,savedir)

In [19]:
print("PSD F1 Score:",PSD_f1Score4)
print("AUC F1 Score:",AUC_f1Score4)
print("AP F1 Score:",AP_f1Score4)

PSD F1 Score: 0.731523903937697
AUC F1 Score: 0.6863202172597566
AP F1 Score: 0.6792297517297518


In [20]:
print(PSD_cm4)
print(AUC_cm4)
print(AP_cm4)

[[0.54901961 0.31372549 0.1372549 ]
 [0.0546875  0.875      0.0703125 ]
 [0.09090909 0.14141414 0.76767677]]
[[0.5        0.31372549 0.18627451]
 [0.09375    0.765625   0.140625  ]
 [0.08080808 0.11111111 0.80808081]]
[[0.69607843 0.14705882 0.15686275]
 [0.2734375  0.625      0.1015625 ]
 [0.16161616 0.11111111 0.72727273]]
